In [1]:
from utils import *
import copy

# Parsing

In [2]:
with open("data/input22.txt") as f:
    data = f.read().rstrip("\n")

class Cube:
    def __init__(self, minpos, maxpos, id) -> None:
        self.xmin, self.ymin, self.zmin = minpos
        self.xmax, self.ymax, self.zmax = maxpos
        self.id = id

def parse(inlines):
    cubes = [(ints(l.split("~")[0]), ints(l.split("~")[1])) for l in inlines.split("\n")]
    out = {}
    for i,c in enumerate(cubes):
        if c[1][2] in out:
            out[c[1][2]].append(Cube((c[0][0]-1, c[0][1]-1, c[0][2]-1), (c[1][0], c[1][1], c[1][2]), i))
        else:
            out[c[1][2]] = [Cube((c[0][0]-1, c[0][1]-1, c[0][2]-1), (c[1][0], c[1][1], c[1][2]), i)]
    return out, len(cubes)

cubes, ncubes = parse(data)

# Part 1

In [3]:
def overlap_horizontal(c1,c2):
    return not (c1.xmin>=c2.xmax or c2.xmin>=c1.xmax or c2.ymax<=c1.ymin or c1.ymax<=c2.ymin)

def fall_down(cubes):
    ground = 0

    anymoved = True
    curcubes = copy.deepcopy(cubes)
    while anymoved:
        anymoved = False
        newcubes = {}
        for _,cubs in curcubes.items():
            for c in cubs:
                hassupport = False
                if c.zmin==ground:
                    hassupport = True
                elif c.zmin in curcubes:
                    for c2 in curcubes[c.zmin]:
                        if overlap_horizontal(c, c2):
                            hassupport = True
                cnew = copy.deepcopy(c)
                if not hassupport:
                    cnew.zmin -= 1
                    cnew.zmax -= 1
                    anymoved = True
                if cnew.zmax in newcubes:
                    newcubes[cnew.zmax].append(cnew)
                else:
                    newcubes[cnew.zmax] = [cnew]
        curcubes = newcubes
    cubes_by_ceil = curcubes
    cubes_by_floor = {}
    for _,cubs in curcubes.items():
        for c in cubs:
            if c.zmin in cubes_by_floor:
                cubes_by_floor[c.zmin].append(copy.deepcopy(c))
            else:
                cubes_by_floor[c.zmin] = [copy.deepcopy(c)]
    return cubes_by_ceil, cubes_by_floor

def has_integrity(cubes_by_ceil, cubes_by_floor, blocked_id):
    zmax_blocked = -1
    for _,cubs in cubes_by_ceil.items():
        for c1 in cubs:
            if c1.id==blocked_id:
                zmax_blocked = c1.zmax
    if zmax_blocked in cubes_by_floor:
        for c1 in cubes_by_floor[zmax_blocked]:
            c1support = False
            if c1.zmin==0:
                c1support = True
            else:
                for c2 in cubes_by_ceil[c1.zmin]:
                    if c2.id!=blocked_id and overlap_horizontal(c1,c2):
                        c1support = True
                        break
            if not c1support:
                return 0
    return 1

def count_disintegrated(cubes_by_ceil, cubes_by_floor, ncubes):
    accum = 0
    for i in range(ncubes):
        accum += has_integrity(cubes_by_ceil, cubes_by_floor, i)
    return accum

cubes_by_ceil, cubes_by_floor = fall_down(cubes)

In [4]:
print(count_disintegrated(cubes_by_ceil, cubes_by_floor, ncubes))

465


# Part 2

In [5]:
def will_fall(cubes_by_ceil, cubes_by_floor, blocked_id):
    zmax_blocked = -1
    cblocked = None
    for _,cubs in cubes_by_ceil.items():
        for c1 in cubs:
            if c1.id==blocked_id:
                zmax_blocked = c1.zmax
                cblocked = c1
    causes_fall = False
    if zmax_blocked in cubes_by_floor:
        for c1 in cubes_by_floor[zmax_blocked]:
            c1support = False
            if c1.zmin==0:
                c1support = True
            else:
                for c2 in cubes_by_ceil[c1.zmin]:
                    if c2.id!=blocked_id and overlap_horizontal(c1,c2):
                        c1support = True
                        break
            if not c1support:
                causes_fall = True
                break
    if causes_fall:
        fallen_ids = {cblocked.id}
        cubes_cur = [cblocked]
        while cubes_cur:
            cubes_new = []
            for curcube in cubes_cur:
                if curcube.zmax in cubes_by_floor:
                    for c3 in cubes_by_floor[curcube.zmax]:
                        if overlap_horizontal(curcube, c3):
                            if c3.id not in fallen_ids:
                                will_fall = True
                                for c4 in cubes_by_ceil[c3.zmin]:
                                    if c4.id not in fallen_ids and overlap_horizontal(c4, c3):
                                        will_fall = False
                                if will_fall:
                                    cubes_new.append(c3)
                                    fallen_ids.add(c3.id)
            cubes_cur = cubes_new
        return len(fallen_ids)-1
    return 0

def count_fallen(cubes_by_ceil, cubes_by_floor, ncubes):
    accum = 0
    for i in range(ncubes):
        accum += will_fall(cubes_by_ceil, cubes_by_floor, i)
    return accum

print(count_fallen(cubes_by_ceil, cubes_by_floor, ncubes))

79042


# Overkill

In [7]:
# import matplotlib.pyplot as plt

# fig,ax = plt.subplots(figsize=(5,20))
# for _,cubs in cubes.items():
#     for c in cubs:
#         ax.plot([c.xmin,c.xmin,c.xmax,c.xmax,c.xmin], [c.zmin,c.zmax,c.zmax,c.zmin,c.zmin])
# plt.show()
# fig,ax = plt.subplots(figsize=(5,20))
# for _,cubs in cubes.items():
#     for c in cubs:
#         ax.plot([c.ymin,c.ymin,c.ymax,c.ymax,c.ymin], [c.zmin,c.zmax,c.zmax,c.zmin,c.zmin])
# plt.show()

# fig,ax = plt.subplots(figsize=(5,20))
# for _,cubs in cubes_by_ceil.items():
#     for c in cubs:
#         ax.plot([c.xmin,c.xmin,c.xmax,c.xmax,c.xmin], [c.zmin,c.zmax,c.zmax,c.zmin,c.zmin])
# plt.show()
# fig,ax = plt.subplots(figsize=(5,20))
# for _,cubs in cubes_by_ceil.items():
#     for c in cubs:
#         ax.plot([c.ymin,c.ymin,c.ymax,c.ymax,c.ymin], [c.zmin,c.zmax,c.zmax,c.zmin,c.zmin])
# plt.show()